In [1]:
import pymysql
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from Cloud_Rules import GameRules
from Cloud_LDA_adaptive import aLDA

In [2]:
"engr_coaptengr"
db = pymysql.connect(host='34.135.162.65',user='coapt_admin',password="coaptengr",database='research_app')
cursor = db.cursor()
cursor.execute("SELECT * FROM in_the_zone_event_log_PLP")

df = pd.DataFrame(cursor.fetchall())
df = df[df[2] == 'CC0617']
df.head()

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
86,87,2022-11-09 17:43:22,CC0617,1668015817,529,beginner,18,0,26.5492,0.1667,...,255,255,255,255,255,255,255,255,255,255
95,96,2022-11-15 16:26:55,CC0617,1668529612,388,beginner,18,18,180.5216,1.0000,...,255,255,255,255,255,255,255,255,255,255
97,98,2022-11-15 16:38:10,CC0617,1668530288,529,intermediate,18,5,80.5364,0.2778,...,255,255,255,255,255,255,255,255,255,255
98,99,2022-11-15 16:43:44,CC0617,1668530622,529,beginner,18,18,162.2751,1.0000,...,255,255,255,255,255,255,255,255,255,255
99,100,2022-11-15 17:21:22,CC0617,1668532879,529,beginner,18,12,151.8852,0.6667,...,255,255,255,255,255,255,255,255,255,255


In [3]:
# event id: 96
cursor.execute("SELECT * FROM in_the_zone_trial_log_PLP")

df = pd.DataFrame(cursor.fetchall(), columns=['trial_id', 'commandtime', 'in_the_zone_event_id', 'username', 'trial_num',
                                              'target_MID', 'timeout', 'zone_dwell_time', 'zone_target_speed', 
                                              'target_zone_tolerance', 'completion', 'completion_time', 'control_efficiency', 
                                              'num_overshoots', 'reaction_time'])
df = df[df['username'] == 'CC0617']
df = df[df['in_the_zone_event_id'] == 96]
trials = df[['trial_num', 'target_MID']].copy().to_numpy()
print(trials)

[[ 1  8]
 [ 2 11]
 [ 3 12]
 [ 4 17]
 [ 5 11]
 [ 6  9]
 [ 7  9]
 [ 8 18]
 [ 9 18]
 [10  9]
 [11  8]
 [12 11]
 [13  8]
 [14 12]
 [15 17]
 [16 17]
 [17 18]
 [18 12]]


In [4]:
cursor.execute("SELECT * FROM in_the_zone_data_log_PLP")

df = pd.DataFrame(cursor.fetchall(), columns=['commandnum', 'commandtime', 'in_the_zone_event_id', 'username', 'trial_num', 
                                              'play_time_elapsed', 'pre_ramp_speed', 'post_ramp_speed', 'smoothed_speed',
                                              'pre_ramp_MID', 'post_ramp_MID', 'smoothed_MID', 'move_timer', 'zone_fill',
                                              'state', 'mav1', 'mav2', 'mav3', 'mav4', 'mav5', 'mav6', 'mav7', 'mav8', 
                                              'lift_off_mask'])
df = df[df['username'] == 'CC0617']
df = df[df['in_the_zone_event_id'] == 96]
df = df[df['state'] != 0]
df.head()

,commandnum,commandtime,in_the_zone_event_id,username,trial_num,play_time_elapsed,pre_ramp_speed,post_ramp_speed,smoothed_speed,pre_ramp_MID,...,state,mav1,mav2,mav3,mav4,mav5,mav6,mav7,mav8,lift_off_mask
91783,91784,2022-11-15 16:26:55,96,CC0617,1,0.0000,0,0,0,0,...,1,864.0,714.0,781.0,833.0,2832.0,2930.0,736.0,1088.0,0
91784,91785,2022-11-15 16:26:55,96,CC0617,1,0.0000,0,0,0,0,...,3,926.0,666.0,794.0,851.0,2948.0,3034.0,816.0,1171.0,0
91785,91786,2022-11-15 16:26:55,96,CC0617,1,0.0000,38,0,0,12,...,3,880.0,591.0,711.0,794.0,2820.0,2935.0,856.0,1133.0,0
91786,91787,2022-11-15 16:26:55,96,CC0617,1,0.0000,38,5,0,12,...,3,889.0,562.0,661.0,802.0,2831.0,2918.0,956.0,1137.0,0
91787,91788,2022-11-15 16:26:55,96,CC0617,1,0.3008,0,10,0,0,...,3,872.0,538.0,646.0,769.0,2812.0,2918.0,935.0,1081.0,0


In [5]:
targetClass = []
for i in range(len(df)):
    targetClass.append(0)
df['targetClass'] = targetClass
df.head()

,commandnum,commandtime,in_the_zone_event_id,username,trial_num,play_time_elapsed,pre_ramp_speed,post_ramp_speed,smoothed_speed,pre_ramp_MID,...,mav1,mav2,mav3,mav4,mav5,mav6,mav7,mav8,lift_off_mask,targetClass
91783,91784,2022-11-15 16:26:55,96,CC0617,1,0.0000,0,0,0,0,...,864.0,714.0,781.0,833.0,2832.0,2930.0,736.0,1088.0,0,0
91784,91785,2022-11-15 16:26:55,96,CC0617,1,0.0000,0,0,0,0,...,926.0,666.0,794.0,851.0,2948.0,3034.0,816.0,1171.0,0,0
91785,91786,2022-11-15 16:26:55,96,CC0617,1,0.0000,38,0,0,12,...,880.0,591.0,711.0,794.0,2820.0,2935.0,856.0,1133.0,0,0
91786,91787,2022-11-15 16:26:55,96,CC0617,1,0.0000,38,5,0,12,...,889.0,562.0,661.0,802.0,2831.0,2918.0,956.0,1137.0,0,0
91787,91788,2022-11-15 16:26:55,96,CC0617,1,0.3008,0,10,0,0,...,872.0,538.0,646.0,769.0,2812.0,2918.0,935.0,1081.0,0,0


In [6]:
index = 0
for i in trials[:,0]:
    df.loc[df["trial_num"] == i, "targetClass"] = trials[index, 1]
    index += 1
df.head()

,commandnum,commandtime,in_the_zone_event_id,username,trial_num,play_time_elapsed,pre_ramp_speed,post_ramp_speed,smoothed_speed,pre_ramp_MID,...,mav1,mav2,mav3,mav4,mav5,mav6,mav7,mav8,lift_off_mask,targetClass
91783,91784,2022-11-15 16:26:55,96,CC0617,1,0.0000,0,0,0,0,...,864.0,714.0,781.0,833.0,2832.0,2930.0,736.0,1088.0,0,8
91784,91785,2022-11-15 16:26:55,96,CC0617,1,0.0000,0,0,0,0,...,926.0,666.0,794.0,851.0,2948.0,3034.0,816.0,1171.0,0,8
91785,91786,2022-11-15 16:26:55,96,CC0617,1,0.0000,38,0,0,12,...,880.0,591.0,711.0,794.0,2820.0,2935.0,856.0,1133.0,0,8
91786,91787,2022-11-15 16:26:55,96,CC0617,1,0.0000,38,5,0,12,...,889.0,562.0,661.0,802.0,2831.0,2918.0,956.0,1137.0,0,8
91787,91788,2022-11-15 16:26:55,96,CC0617,1,0.3008,0,10,0,0,...,872.0,538.0,646.0,769.0,2812.0,2918.0,935.0,1081.0,0,8


In [9]:
adapted_err = []
adapted_wrules_err = []

class_data = df[['smoothed_MID', 'targetClass', 'mav1', 'mav2', 'mav3', 'mav4', 'mav5', 'mav6', 'mav7', 'mav8']].to_numpy()

""" Adapted """ 
X = adapt_sample[:, :-1]
y = adapt_sample[:, -1]

alda_norules = aLDA()
prev_means_norules, prev_covs_norules = alda_norules.fit(X, y, flag=0)

X = test_sample[:, :-1]
y = test_sample[:, -1]
alda_norules_preds = alda_norules.predict(X)

print(f"First game adapted: {np.around((y!=alda_norules_preds).sum() / y.size, 2) * 100}%")

adapted_err.append(np.around((y!=alda_norules_preds).sum() / y.size, 2) * 100)

""" Adapted with rules """
rules = GameRules()
new_data = rules.augmented_rule(df)
new_data = np.vstack((new_data))[:,:-1]

X = new_data[:, 2:]
y = new_data[:, 1]

np.random.seed(42)
combined = np.hstack((X,y.reshape(len(y),1)))
np.random.shuffle(combined)
adapt_sample = combined[:len(y)//2, :]
test_sample = combined[len(y)//2:, :]

X = adapt_sample[:, :-1]
y = adapt_sample[:, -1]

alda = aLDA()
prev_means, prev_covs = alda.fit(X, y, flag=0)

X = test_sample[:, :-1]
y = test_sample[:, -1]
alda_preds = alda.predict(X)

# print_array(y)
# print_array(alda_preds)
print(f"First game adapted w/ rules: {np.around((y!=alda_preds).sum() / y.size, 2) * 100}%")

adapted_wrules_err.append(np.around((y!=alda_preds).sum() / y.size, 2) * 100)

First game adapted: 38.0%
First game adapted w/ rules: 38.0%


In [ ]:
""" To create and upload a new table
import sqlite3

conn = sqlite3.connect('test_database') 
c = conn.cursor()

c.execute('''
          CREATE TABLE IF NOT EXISTS products
          ([product_id] INTEGER PRIMARY KEY, [product_name] TEXT, [price] INTEGER)
          ''')
          
c.execute('''
          INSERT INTO products (product_id, product_name, price)

                VALUES
                (1,'Computer',800),
                (2,'Printer',200),
                (3,'Tablet',300),
                (4,'Desk',450),
                (5,'Chair',150)
          ''')                     

conn.commit()
"""